#### Problem 5: Multiagent Bidding Strategy

In [15]:
import pandas as pd

#for multi-agent we used only validation dataset
validation_df = pd.read_csv('validation.csv')    

validation_df.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,4,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,windows_ie,211.144.203.*,79,79,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,23,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10111"
1,0,1,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,windows_chrome,58.247.250.*,79,79,1.0,...,90,0,1,0,c938195f9e404b4f38c7e71bf50263e5,238,75,a3becb967c015e9acb5e72f22b100ef8,3476,"10075,10057,10024,10052,10083,10063,10006,10110"
2,0,4,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,windows_ie,117.12.111.*,2,2,2.0,...,250,2,0,5,911b2d84826786018761e8c0b0a3a60c,238,65,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10024,10059,10063,13776,10083,10006,10111"
3,0,5,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,windows_chrome,61.187.224.*,201,205,2.0,...,280,2,0,5,dc0998c10f8f0b623b5d949e8272e4c7,238,6,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10057,10006,10063,10110"
4,0,1,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,windows_chrome,117.41.145.*,134,135,2.0,...,200,1,0,5,23485fcd23122d755d38f8c89d46ca56,249,5,7e0e909ea89f707bc20f9585c9072081,3476,"10006,10063,13776,10110"


In [16]:
#only required columns

validation_df_part = validation_df[['payprice', 'click']]

validation_df_part.shape

(303925, 2)

In [3]:
#case with two agents, one is using Constant bidding (80) and the second one is using Random bidding in a range (20, 115)
#80 and 20-115 range was determined in the basic bidding strategies section before

import numpy as np
from random import randrange

results = np.empty((2, 5), dtype=object) 

budget_agent1 = 6250000
budget_agent2 = 6250000

#---
clicks1 = clicks2 = 0
impressions1 = impressions2 = 0
criteria1 = criteria2 = 0
#---

for i in range(0, len(validation_df_part)):   
    
    agent1 = 80                         # from constant bidding strategy
    agent2 = randrange(20, 115)         # from random bidding strategy
    
    if agent1 > agent2:
        if budget_agent1 > agent1:
            impressions1 += 1 
            if agent1 > (validation_df_part['payprice'].values[i]):
                clicks1 += validation_df_part['click'].values[i]
                criteria1 = criteria1 + 1   
                budget_agent1 = budget_agent1 - max(validation_df_part['payprice'].values[i], agent2) #second highest price
                                
    else:
        if budget_agent2 > agent2:
            impressions2 += 1 
            if agent2 > (validation_df_part['payprice'].values[i]):
                clicks2 += validation_df_part['click'].values[i]
                criteria2 = criteria2 + 1   
                budget_agent2 = budget_agent2 - max(validation_df_part['payprice'].values[i], agent1)
            
results[0, 0] = 'Agent 1'
results[0, 1] = clicks1
results[0, 2] = impressions1
results[0, 3] = (6250000 - budget_agent1)/1000
results[0, 4] = criteria1


results[1, 0] = 'Agent 2'
results[1, 1] = clicks2                      
results[1, 2] = impressions2    
results[1, 3] = (6250000 - budget_agent2)/1000    
results[1, 4] = criteria2   

results

array([['Agent 1', 53, 168188, 6249.98, 107471],
       ['Agent 2', 34, 104465, 6249.956, 77216]], dtype=object)

from the results it can be seen that Agent1 gets more clicks mostly because his price is 80 and the range for price for Agent2 is between 20 and 115 (in ~80% of cases random function generates numbers less than 80). The ranges for Agent2 were chosen without knowing the biding strategy of Agent1. Now, let's assume that Agent2 knows Agent1's constant bids and he adapted his ranges (new Grip search) based on this knowledge. 

ps. There is a high factor of randomness. And each run shows different results

In [18]:
# del results, clicks1, clicks2, impressions1, impressions2, criteria1, criteria2, budget_agent1, budget_agent2

In [17]:
### chosing new ranges for Agent2 only

results = np.empty((120,7), dtype=object)   

n=0  # counter

for lowerbound in range(0,300,20):         #the step was increased to 20 in order to run faster   
    for upperbound in range(5,305,20):     #the step was increased to 20 in order to run faster  
        impressions = 0
        clicks = 0
        budget = 6250000
        CTR = 0
        criteria1 = 0
      
        if lowerbound < upperbound:
            for i in range(0, len(validation_df_part)):   
                j = randrange(lowerbound, upperbound)
                if budget > j:
                    impressions += 1
                    if (j > (validation_df_part['payprice'].values[i])) or (j > 80):            #added constant bidding
                        clicks += validation_df_part['click'].values[i]
                        criteria1 = criteria1 + 1   # number of won impressions based on criteria #1
                        budget = budget - validation_df_part['payprice'].values[i]
            
            results[n, 0] = lowerbound                     
            results[n, 1] = upperbound                     
            results[n, 2] = impressions                    
            results[n, 3] = clicks                         
            results[n, 4] = round(clicks / impressions,6)  
            results[n, 5] = criteria1                      
            results[n, 6] = (6250000 - budget)/1000            
       
            n = n+1 

In [19]:
np.set_printoptions(threshold=np.inf)

results[results[:,3].argsort()[::-1]] 

array([[20, 145, 303885, 79, 0.00026, 120443, 6249.936],
       [20, 185, 234929, 71, 0.000302, 112082, 6249.947],
       [60, 145, 206514, 69, 0.000334, 120797, 6249.944],
       [40, 165, 223438, 69, 0.000309, 115994, 6249.949],
       [0, 205, 231301, 65, 0.000281, 107823, 6249.958],
       [80, 85, 259483, 65, 0.00025, 139549, 6249.964],
       [0, 225, 226555, 64, 0.000282, 104243, 6249.929],
       [80, 105, 188099, 64, 0.00034, 130803, 6249.92],
       [80, 285, 108540, 62, 0.000571, 96340, 6249.975],
       [80, 165, 148859, 61, 0.00041, 116207, 6249.919],
       [20, 225, 205045, 61, 0.000297, 104114, 6249.93],
       [100, 105, 164114, 61, 0.000372, 124972, 6249.902],
       [60, 165, 181903, 61, 0.000335, 116314, 6249.98],
       [140, 225, 105340, 60, 0.00057, 96021, 6249.941],
       [180, 185, 103937, 60, 0.000577, 95489, 6249.872],
       [80, 125, 169650, 60, 0.000354, 125272, 6249.92],
       [160, 245, 97205, 59, 0.000607, 91258, 6249.867],
       [100, 225, 116077, 5

it can be seen that the range is increased slightly. Let's use new ranges and replay the game. In this case the situation for Agent2 should be better, but there is a high randomness which we have to take always into account

In [21]:
# second run, 2 agents, but agent with random bidding strategy adapted to constant

results = np.empty((2, 5), dtype=object) 

budget_agent1 = 6250000
budget_agent2 = 6250000

#---
clicks1 = clicks2 = 0
impressions1 = impressions2 = 0
criteria1 = criteria2 = 0
#---

for i in range(0, len(validation_df_part)):   
    
    agent1 = 80
    agent2 = randrange(20, 145)  # the range increased from 20-105 to 20-145
    
    if agent1 > agent2:
        if budget_agent1 > agent1:
            impressions1 += 1 
            if agent1 > (validation_df_part['payprice'].values[i]):
                clicks1 += validation_df_part['click'].values[i]
                criteria1 = criteria1 + 1   
                budget_agent1 = budget_agent1 - max(validation_df_part['payprice'].values[i], agent2) 
                                
    else:
        if budget_agent2 > agent2:
            impressions2 += 1 
            if agent2 > (validation_df_part['payprice'].values[i]):
                clicks2 += validation_df_part['click'].values[i]
                criteria2 = criteria2 + 1   
                budget_agent2 = budget_agent2 - max(validation_df_part['payprice'].values[i], agent1)
            
results[0, 0] = 'Agent 1'
results[0, 1] = clicks1
results[0, 2] = impressions1
results[0, 3] = (6250000 - budget_agent1)/1000
results[0, 4] = criteria1


results[1, 0] = 'Agent 2'
results[1, 1] = clicks2                      
results[1, 2] = impressions2    
results[1, 3] = (6250000 - budget_agent2)/1000    
results[1, 4] = criteria2

results

array([['Agent 1', 40, 145830, 5421.116, 93250],
       ['Agent 2', 45, 97837, 6249.962, 75955]], dtype=object)

Agent 2 in this case did a bit better comparing with Agent 1. Let's adapt Agent 1 as well based on Linear search, and we assume that Agent 1 knows that Agent 2 bet randomly in a range between 20 and 145

In [26]:
del results

results = np.empty((300, 5), dtype=object)  

for j in range (1,301): 
    
    impressions = 0
    clicks = 0
    budget = 6250000    
    CTR = 0
    
    for i in range(0, len(validation_df_part)):      
        if budget > j:
            impressions += 1
            if (j > (validation_df_part['payprice'].values[i])) or (j > randrange(20, 145)):
                clicks += validation_df_part['click'].values[i]
                budget = budget - validation_df_part['payprice'].values[i]
        else: 
            break
            
    results[j-1, 0] = j                                  
    results[j-1, 1] = impressions                        
    results[j-1, 2] = clicks                             
    results[j-1, 3] = round(clicks / impressions,6)      
    results[j-1, 4] = (6250000 - budget)/1000            

In [27]:
#optimal new constant strategy

results[results[:,2].argsort()[::-1]]

array([[110, 215682, 77, 0.000357, 6249.969],
       [90, 303925, 72, 0.000237, 5867.57],
       [105, 235978, 70, 0.000297, 6249.919],
       [97, 277389, 70, 0.000252, 6249.917],
       [104, 240633, 69, 0.000287, 6249.915],
       [109, 219983, 69, 0.000314, 6249.899],
       [92, 303925, 68, 0.000224, 6173.406],
       [115, 198752, 68, 0.000342, 6249.947],
       [93, 302412, 67, 0.000222, 6249.914],
       [94, 297397, 67, 0.000225, 6249.946],
       [114, 202191, 67, 0.000331, 6249.887],
       [101, 255992, 67, 0.000262, 6249.902],
       [106, 232074, 67, 0.000289, 6249.921],
       [111, 213066, 66, 0.00031, 6249.911],
       [91, 303925, 66, 0.000217, 6018.539],
       [98, 272691, 66, 0.000242, 6249.917],
       [88, 303925, 65, 0.000214, 5370.219],
       [86, 303925, 65, 0.000214, 5096.489],
       [103, 243988, 64, 0.000262, 6249.923],
       [133, 151527, 63, 0.000416, 6249.925],
       [123, 175454, 63, 0.000359, 6249.899],
       [108, 224691, 63, 0.00028, 6249.931],


the price for Agent 1 increased from 80 to 100. Let's replay the game and see how the increase just to ten points will change the situation

In [29]:
# del results, clicks1, clicks2, impressions1, impressions2, criteria1, criteria2, budget_agent1, budget_agent2

In [31]:
# 3rd run

results = np.empty((2, 5), dtype=object) 

budget_agent1 = 6250000
budget_agent2 = 6250000

#---
clicks1 = clicks2 = 0
impressions1 = impressions2 = 0
criteria1 = criteria2 = 0
#---

for i in range(0, len(validation_df_part)):   
    
    agent1 = 110   # now it is 90 not 80
    agent2 = randrange(20, 145)
    
    if agent1 > agent2:
        if budget_agent1 > agent1:
            impressions1 += 1 
            if agent1 > (validation_df_part['payprice'].values[i]):
                clicks1 += validation_df_part['click'].values[i]
                criteria1 = criteria1 + 1   
                budget_agent1 = budget_agent1 - max(validation_df_part['payprice'].values[i], agent2) #second highest price
                                
    else:
        if budget_agent2 > agent2:
            impressions2 += 1 
            if agent2 > (validation_df_part['payprice'].values[i]):
                clicks2 += validation_df_part['click'].values[i]
                criteria2 = criteria2 + 1   
                budget_agent2 = budget_agent2 - max(validation_df_part['payprice'].values[i], agent1)
            
results[0, 0] = 'Agent 1'
results[0, 1] = clicks1
results[0, 2] = impressions1
results[0, 3] = (6250000 - budget_agent1)/1000
results[0, 4] = criteria1


results[1, 0] = 'Agent 2'
results[1, 1] = clicks2                      
results[1, 2] = impressions2    
results[1, 3] = (6250000 - budget_agent2)/1000    
results[1, 4] = criteria2

results

array([['Agent 1', 40, 107838, 6249.975, 84195],
       ['Agent 2', 31, 69371, 6249.904, 56593]], dtype=object)

In [32]:
del results, clicks1, clicks2, impressions1, impressions2, criteria1, criteria2, budget_agent1, budget_agent2

The situation for Agent 1 is changed again. Until now we did everything manually, which is good to see the changes, but now, let's run the game 10 times automatically, and after each game we will record results and the agent with less clicks will adapt his strategy to new situation. Kind of repeated simulation of bidding between 2 agents - 10 times. 

In [17]:
#function which accepts lowerbound and upperbound from random biddnig as input and generates new constant bid

def adapt_for_constant (lb, ub):           # lb, ub - lowerbound and upperbound 
    
    results = np.empty((300, 3), dtype=object)  
    r = np.empty((300, 3), dtype=object)    

    for j in range (1,301): 
    
        clicks = 0
        budget = 6250000    
    
        for i in range(0, len(validation_df_part)):      
            if budget > j:
                if (j > (validation_df_part['payprice'].values[i])) or (j > randrange(lb, ub)):
                    clicks += validation_df_part['click'].values[i]
                    budget = budget - validation_df_part['payprice'].values[i]
                else: 
                    break
            
        results[j-1, 0] = j                                  
        results[j-1, 1] = clicks                             
        results[j-1, 2] = (6250000 - budget)/1000          
  
        if j == 300: 
            r = results[results[:,1].argsort()[::-1]]
            
            return r[0,0]

In [18]:
#function which accepts constant value from const biddnig as input and generates new ranges for random bidding

def adapt_for_random (constpr):

    results = np.empty((120,4), dtype=object)   

    n=0  # counter

    for lowerbound in range(0,300,20):            
        for upperbound in range(5,305,20):       
            
            clicks = 0
            budget = 6250000
            
            if lowerbound < upperbound:
                for i in range(0, len(validation_df_part)):   
                    j = randrange(lowerbound, upperbound)
                    
                    if budget > j:
                        if (j > (validation_df_part['payprice'].values[i])) and (j > constpr):          
                            clicks += validation_df_part['click'].values[i]
                            budget = budget - validation_df_part['payprice'].values[i]

                results[n, 0] = lowerbound                     
                results[n, 1] = upperbound                     
                results[n, 2] = clicks                         
                results[n, 3] = (6250000 - budget)/1000            

                n = n+1
                
    return results[results[:,2].argsort()[::-1]][0,0], results[results[:,2].argsort()[::-1]][0,1]

In [7]:
#just to check

adapt_for_random(90)

(20, 145)

In [23]:
# 10 runs
import numpy as np
from random import randrange

results = np.empty((5, 9), dtype=object) 

a1 = 80   #from the constant bidding strategy
a2 = 10   #lower bound from the random bidding strategy
a3 = 115  #upper bound from the random bidding strategy   

for x in range(1,11):             #number of runs

    budget_agent1 = budget_agent2 = 6250000   
    clicks1 = clicks2 = 0
    impressions1 = impressions2 = 0
    criteria1 = criteria2 = 0
    
    for i in range(0, len(validation_df_part)):   

        if x == 1:
            
            agent1 = a1   
            agent2 = randrange(a2, a3)
                
            if agent1 > agent2:
                if budget_agent1 > agent1:
                    impressions1 += 1 
                    if agent1 > (validation_df_part['payprice'].values[i]):
                        clicks1 += validation_df_part['click'].values[i]
                        criteria1 = criteria1 + 1   
                        budget_agent1 = budget_agent1 - max(validation_df_part['payprice'].values[i], agent2)

            else:
                if budget_agent2 > agent2:
                    impressions2 += 1 
                    if agent2 > (validation_df_part['payprice'].values[i]):
                        clicks2 += validation_df_part['click'].values[i]
                        criteria2 = criteria2 + 1   
                        budget_agent2 = budget_agent2 - max(validation_df_part['payprice'].values[i], agent1)
        else: 
            
            agent1 = a1   
            agent2 = randrange(a2, a3)
            
            if agent1 > agent2:
                if budget_agent1 > agent1:
                    impressions1 += 1 
                    if agent1 > (validation_df_part['payprice'].values[i]):
                        clicks1 += validation_df_part['click'].values[i]
                        criteria1 = criteria1 + 1   
                        budget_agent1 = budget_agent1 - max(validation_df_part['payprice'].values[i], agent2)

            else:
                if budget_agent2 > agent2:
                    impressions2 += 1 
                    if agent2 > (validation_df_part['payprice'].values[i]):
                        clicks2 += validation_df_part['click'].values[i]
                        criteria2 = criteria2 + 1   
                        budget_agent2 = budget_agent2 - max(validation_df_part['payprice'].values[i], agent1)
 
    results[x-1, 0] = 'Round '+ str(x)
    
    results[x-1, 1] = 'Agent 1'
    results[x-1, 2] = clicks1
    results[x-1, 3] = impressions1
    results[x-1, 4] = (6250000 - budget_agent1)/1000

    results[x-1, 5] = 'Agent 2'
    results[x-1, 6] = clicks2                      
    results[x-1, 7] = impressions2    
    results[x-1, 8] = (6250000 - budget_agent2)/1000    
       
    if clicks1 > clicks2:
        a2,a3 = adapt_for_random(a1)
    
    else:
        a1 = adapt_for_constant(a2,a3)
         

In [24]:
#on last steps the price for constant and ranges for random bidding strategies go extremely high

results

array([['Round 1', 'Agent 1', 61, 174552, 6249.988, 'Agent 2', 31,
        101148, 6036.084],
       ['Round 2', 'Agent 1', 49, 145680, 5434.107, 'Agent 2', 38, 97912,
        6249.953],
       ['Round 3', 'Agent 1', 52, 142517, 5797.63, 'Agent 2', 37, 102417,
        6249.949],
       ['Round 4', 'Agent 1', 0, 0, 0.0, 'Agent 2', 34, 117288, 6249.981],
       ['Round 5', 'Agent 1', 42, 60526, 6249.775, 'Agent 2', 0, 0, 0.0]],
      dtype=object)